In [73]:
import pandas as pd

In [163]:
%matplotlib inline

In [74]:
df2013 = pd.read_excel('./wszystkiejednostki.xlsx', header = [1])
df2013 = df2013[~(df2013['Nazwa jednostki'].isnull())].set_index('Nazwa jednostki')
del df2013.index.name

In [75]:
df2017 = pd.read_excel('./ca19d390d520ad9a37d6f2ba7c8499e4.xlsx')
df2017 = df2017[~(df2017['Instytucja'].isnull())].set_index('Instytucja')
del df2017.index.name

In [76]:
both = []
for x in df2017.index:
    if x in df2013.index:
        both.append(x)

In [90]:
idx2017 = [(x.replace('\n', ' ').strip().lower(), x) for x in df2017.index]
idx2013 = [(x.replace('\n', ' ').strip().lower(), x) for x in df2013.index]
len(set(idx2013))==len(idx2013), len(set(idx2017))==len(idx2017)
idx2017 = dict(idx2017)
idx2013 = dict(idx2013)

In [144]:
rev_idx2017 = {v: k for k,v in idx2017.items()}
rev_idx2013 = {v: k for k,v in idx2013.items()}

In [126]:
common_keys = sorted(set(idx2013).intersection(set(idx2017)))

In [129]:
common_2013 = [idx2013[k] for k in common_keys]
common_2017 = [idx2017[k] for k in common_keys]

In [130]:
common_2013[1], common_2017[1]

('Akademia Górniczo-Hutnicza im. Stanisława Staszica w Krakowie; Wydział\nEnergetyki i Paliw',
 'Akademia Górniczo-Hutnicza im. Stanisława Staszica w Krakowie; Wydział Energetyki i Paliw')

In [173]:
columns = ['Osiągnięcia naukowe i twórcze',
'Potencjał naukowy',
'Praktyczne efekty działalności naukowej i artystycznej',
'Pozostałe efekty działalności naukowej i artystycznej',
'Kategoria']

In [174]:
data2017 = df2017.loc[common_2017].iloc[:,3:8].copy()
data2017.index = [rev_idx2017[k] for k in data2017.index]
data2017.columns=columns

In [179]:
data2013 = df2013.loc[common_2013].iloc[:,[3,4,5,6,-1]].copy()
data2013.index = [rev_idx2013[k] for k in data2013.index]
data2013.columns=columns

In [149]:
delta = data2017-data2013

In [186]:
sizes = {'A': 16, 'A+': 20, 'B': 12, 'C': 8}

In [182]:
colors = {'A': '#fc8d62', 'A+': '#66c2a5', 'B': '#8da0cb', 'C': '#e78ac3'}

In [233]:
from itertools import combinations, combinations_with_replacement

In [155]:
kombinacje = list(combinations(delta.columns, 2))

In [187]:
kolory2017=data2017['Kategoria'].apply(lambda x: colors[x])
rozmiary2013=data2013['Kategoria'].apply(lambda x: sizes[x])

In [201]:
from IPython.display import display,HTML
from plotly import __version__
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [202]:
init_notebook_mode(connected=True)

AttributeError: 'Series' object has no attribute 'toarray'

In [234]:
list(combinations_with_replacement(factors.keys() , 2))

[('C', 'C'),
 ('C', 'A+'),
 ('C', 'B'),
 ('C', 'A'),
 ('A+', 'A+'),
 ('A+', 'B'),
 ('A+', 'A'),
 ('B', 'B'),
 ('B', 'A'),
 ('A', 'A')]

In [266]:
!mkdir delta_per_category

In [269]:
for k in kombinacje:
    komb=list(k)
    all_traces = []

    layout= go.Layout(
        title= 'Δ oceny parametrycznej 2017-2013',
        hovermode= 'closest',
        xaxis= dict(
            title= komb[0],
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= komb[1],
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend= True
    )

    for kategoria_in_2013 in ['A+', 'A', 'B', 'C']:
        for kategoria_in_2017 in ['A+', 'A', 'B', 'C']:
            index_kat2013 = data2013[data2013['Kategoria']==kategoria_in_2013].index.tolist()
            index_kat = [x for x in data2017[data2017['Kategoria']==kategoria_in_2017].index.tolist() if x in index_kat2013]

            trace = go.Scatter(
                x = delta.loc[index_kat,komb[0]].tolist(),
                y = delta.loc[index_kat,komb[1]].tolist(),
                name = "{}->{}".format(kategoria_in_2013,kategoria_in_2017),
                text=index_kat,
                marker = dict(
                    size = sizes[kategoria_in_2013],
                    color = colors[kategoria_in_2017],
                ),
                mode = 'markers',
            )
            all_traces.append(trace)

    plot(go.Figure(data=all_traces, layout=layout), filename="./delta_per_category/{}-{}.html".format(komb[0], komb[1]))